In [1]:
!pip install eli5

     |████████████████████████████████| 112kB 4.4MB/s 


In [0]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

from ast import literal_eval
from tqdm import tqdm_notebook

In [3]:
cd "/content/drive/My Drive/Colab Notebook"

/content/drive/My Drive/Colab Notebook


In [4]:
ls

Matrix/


In [5]:
ls Matrix/data

men_shoes.csv


In [0]:
df = pd.read_csv('Matrix/data/men_shoes.csv', low_memory=False)

In [0]:
def run_model(feats, model = DecisionTreeRegressor(max_depth=5)):
  x = df[ feats ].values
  y = df['prices_amountmin'].values

  scores = cross_val_score(model, x, y, scoring = 'neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [12]:
df['brand_cat'] = df['brand'].map(lambda x: str(x).lower()).factorize()[0]
run_model(['brand_cat'])

(-58.133398968282776, 4.206122611474276)

In [10]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat'], model)

(-57.47223572384038, 4.328288468270897)

In [14]:
df.features.values

array(['[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SERVUS BY HONEYWELL"]},{"key":"manufacturer_part_number","value":["ZSR101BLMLG"]}]',
       ...,
       '[{"key":"Gender","value":["Unisex"]},{"key":"Frame Style","value":["Wrap"]},{"key":"Polarized Lenses","value":["Polarized lenses"]},{"key":"Color","value":["Multi-color"]}]',


In [18]:
str_dict = '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]'

literal_eval(str_dict)

[{'key': 'Gender', 'value': ['Men']},
 {'key': 'Shoe Size', 'value': ['M']},
 {'key': 'Shoe Category', 'value': ["Men's Shoes"]},
 {'key': 'Color', 'value': ['Multicolor']},
 {'key': 'Manufacturer Part Number', 'value': ['8190-W-NAVY-7.5']},
 {'key': 'Brand', 'value': ['Josmo']}]

In [0]:
def parse_fetures(x):
  output_dict = {}
  if str(x) == 'nan' : return output_dict
  features =  literal_eval(x.replace('\\"', '"'))
  for item in features:
    key = item['key'].lower().strip()
    value = item['value'][0].lower().strip()
    output_dict[key] = value

  return output_dict
df['features_parsed'] = df['features'].map(parse_fetures)

In [28]:
keys = set()

df['features_parsed'].map( lambda x: keys.update(x.keys()))

len(keys)

476

In [31]:
def get_name_feat(key):
  return 'feat' + key

for key in tqdm_notebook(keys):
  df[get_name_feat(key)] = df.features_parsed.map(lambda feats: feats[key] if key in feats else np.nan)

In [32]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'featairport friendly', 'featuv protection', 'featnavy blue',
       'featparts', 'featfit:', 'feathardsided or softsided', 'featisbn',
       'featchain length', 'featprimary color', 'featfront style'],
      dtype='object', length=526)

In [0]:
keys_stat = {}
for key in keys:
  keys_stat[key] = df [ False == df[get_name_feat(key)].isnull() ].shape[0] / df.shape[0] * 100

In [43]:
{k:v for k,v in keys_stat.items() if v > 30}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
df['feat_brand_cat'] = df['featbrand'].factorize()[0]
df['feat_color_cat'] = df['featcolor'].factorize()[0]
df['feat_gender_cat'] = df['featgender'].factorize()[0]
df['feat_manufacture part number_cat'] = df['featmanufacturer part number'].factorize()[0]
df['feat_material_cat'] = df['featmaterial'].factorize()[0]

df['feat_sport_cat'] = df['featsport'].factorize()[0]
df['feat_style_cat'] = df['featstyle'].factorize()[0]

for key in keys:
  df [get_name_feat(key) + '_cat'] = df[get_name_feat(key)].factorize()[0]

In [54]:
df['brand'] = df['brand'].map(lambda x: str(x).lower() )
df[ df.brand == df.featbrand ][ ['brand', 'featbrand']].shape

(8846, 2)

In [56]:
model = RandomForestRegressor(max_depth = 5, n_estimators = 100)
run_model(['brand_cat'], model)

(-57.29814691463007, 4.223543733102832)

In [82]:
feat_cat = [x for x in df.columns if 'cat' in x]
feat_cat

['categories',
 'brand_cat',
 'featfabrication',
 'featshoe category',
 'featlocation - country',
 'featmulti pack indicator',
 'featcatalog',
 'featrecommended location',
 'featcertifications and listings',
 'featlocation - city/state',
 'featclothing category',
 'feat_brand_cat',
 'feat_color_cat',
 'feat_gender_cat',
 'feat_manufacture part number_cat',
 'feat_material_cat',
 'feat_sport_cat',
 'feat_style_cat',
 'featis portable_cat',
 'featcountry of manufacture_cat',
 'feat100% adidas authentic_cat',
 'featsports team_cat',
 'featexpandable_cat',
 'featlength_cat',
 'featchronograph_cat',
 'featmodel_cat',
 'feathandbag style_cat',
 'featpackageweight_cat',
 'featcase type_cat',
 'featcountry////region of manufacture_cat',
 'featcase diameter_cat',
 'featshape_cat',
 'featwidth_cat',
 'featcompass_cat',
 'featdiameter_cat',
 'featsleeve style_cat',
 'featbridge width_cat',
 'featwatch case shape_cat',
 'featvehicle type_cat',
 'featwash_cat',
 'featresolution_cat',
 'feattoe type

In [0]:
feats = ['brand_cat',
         'feat_brand_cat',
         'featmaterial_cat',
         'feat_material_cat',
         'featweight_cat',
         'featadjustable_cat',
         'featfabric content_cat',
       #  'featresizable_cat',
       #  'featshoe category_cat',
       #  'featfabric material_cat',
       #  'featjacket length_cat',
       #  'featcolor_cat',
       #  'feat_brand_cat',
       #  'featcase thickness_cat',
       #  'featcase diameter_cat',
       # 'featitem package quantity_cat',
       # 'feat_gender_cat'
         ]
 

#  feats += feat_cat
#  feats = list(set(feats))


model = RandomForestRegressor(max_depth = 5, n_estimators = 100)
result = run_model( feats, model)

In [124]:
x = df[ feats ].values
y = df[ 'prices_amountmin'].values

m = RandomForestRegressor(max_depth = 5, n_estimators = 100, random_state = 0)
m.fit(x, y)

print(result)
perm = PermutationImportance(m, random_state = 1).fit(x, y);
eli5.show_weights(perm, feature_names = feats)

(-57.31157785570464, 4.155339007779719)


Weight,Feature
0.2700 ± 0.0099,brand_cat
0.0452 ± 0.0071,featadjustable_cat
0.0440 ± 0.0025,featmaterial_cat
0.0374 ± 0.0030,feat_brand_cat
0.0274 ± 0.0027,feat_material_cat
0.0174 ± 0.0020,featfabric content_cat
0.0126 ± 0.0013,featweight_cat


In [80]:
df [df[ 'brand' ] == 'nike'].features_parsed.sample(5).values

array([{}, {'condition': 'new with box', 'type': 'cleats'},
       {'gender': 'men', 'shoe category': 'basketball', 'fabric content': '100% synthetic', 'color': 'sport royal/game royal', 'model': '314996-401', 'casual & dress shoe style': 'basketball shoes', 'brand': 'nike', 'manufacturer_part_number': '314996-401-9-5'},
       {'main color': 'white', 'type': 'cleats', 'condition': 'new with box'},
       {'sport': 'soccer', 'type': 'cleats', 'condition': 'new with box'}],
      dtype=object)

In [128]:
!git config --global user.email "Czarny_77@onet.pl"
!git config --global user.name "Jakub"

/content/drive/My Drive/Colab Notebook/Matrix
